# Process Design & Simulation: Designing a Flash Drum Separator

**Objective:** This lesson will guide you through a complete, albeit simplified, process design and simulation workflow. We will design a flash drum to separate a binary mixture of benzene and toluene, a classic chemical engineering problem.

**Learning Goals:**
1.  Understand the role of a flash drum in chemical processes.
2.  Apply the principles of Vapor-Liquid Equilibrium (VLE), including Raoult's Law and Antoine's Equation.
3.  Formulate the material and energy balance equations (the "Rachford-Rice" equations) that govern the separation.
4.  Use Python's scientific libraries (`NumPy`, `SciPy`) to solve the system of non-linear equations that describe the process.
5.  Analyze and interpret the simulation results to evaluate the performance of our design.

## The Design Challenge: Flash Separation

Imagine we have a liquid feed stream containing a 50/50 mole mix of benzene and toluene at a high pressure. Our goal is to separate this mixture into a vapor stream enriched in the more volatile component (benzene) and a liquid stream enriched in the less volatile component (toluene).

A **flash drum** is the perfect unit operation for this. The process works by taking the high-pressure liquid feed, passing it through a valve to drop the pressure, and feeding it into a large tank (the drum). The sudden pressure drop causes a portion of the liquid to instantly vaporize or "flash". Because the components have different volatilities, the vapor and liquid phases will have different compositions.

![Flash Drum Diagram](https://upload.wikimedia.org/wikipedia/commons/c/c4/Vap-Liq_Separator.png)

**Our Task:** Given a feed stream and a desired operating pressure and temperature for the drum, we need to calculate:
*   The fraction of the feed that vaporizes ($V/F$).
*   The composition of the resulting vapor stream ($y_i$).
*   The composition of the resulting liquid stream ($x_i$).

## Part 1: The Physics - Vapor-Liquid Equilibrium (VLE)

The separation is only possible because of VLE. For an ideal mixture (which benzene-toluene is a good approximation of), the VLE relationship is described by **Raoult's Law**:
$$ y_i P = x_i P_i^{sat} $$
where:
*   $y_i$ is the mole fraction of component $i$ in the vapor phase.
*   $x_i$ is the mole fraction of component $i$ in the liquid phase.
*   $P$ is the total pressure of the system (the drum pressure).
*   $P_i^{sat}$ is the vapor pressure of pure component $i$ at the system temperature.

We can define an equilibrium constant, $K_i$:
$$ K_i = \frac{y_i}{x_i} = \frac{P_i^{sat}}{P} $$

To calculate the vapor pressure, $P_i^{sat}$, we use the **Antoine Equation**, an empirical formula that relates vapor pressure to temperature:
$$ \log_{10}(P_i^{sat}) = A_i - \frac{B_i}{C_i + T} $$

## Part 2: The Model - Rachford-Rice Flash Equations

To solve for the unknowns in our flash drum, we need to solve a system of equations based on material balances.

Let:
*   $F$ = Molar flow rate of the feed (we'll assume $F=1$ mol/s).
*   $L$ = Molar flow rate of the liquid product.
*   $V$ = Molar flow rate of the vapor product.
*   $z_i$ = Mole fraction of component $i$ in the feed.

**Overall Material Balance:**
$$ F = L + V $$

**Component Material Balance:**
$$ F z_i = L x_i + V y_i $$

By combining these balances with the VLE relationship ($y_i = K_i x_i$), we can derive the famous **Rachford-Rice equation**:
$$ \sum_{i} \frac{z_i (K_i - 1)}{1 + \psi (K_i - 1)} = 0 $$
where $\psi$ is the vapor fraction, $V/F$. This single, powerful equation allows us to solve for the fraction of the feed that vaporizes. We will solve this numerically.

## Part 3: Python Implementation

Let's translate this theory into a working simulation.

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.optimize import fsolve # A numerical solver for non-linear equations
import matplotlib.pyplot as plt

# --- Define System Parameters ---

# Feed Conditions
F = 1.0           # Molar flow rate of feed (mol/s)
z_B = 0.5         # Mole fraction of Benzene in feed
z_T = 0.5         # Mole fraction of Toluene in feed
z = np.array([z_B, z_T])

# Flash Drum Operating Conditions
P = 1.2           # Drum pressure (atm)
T = 95            # Drum temperature (Celsius)

# Antoine Coefficients for P_sat in mmHg and T in Celsius
# (Source: Perry's Chemical Engineers' Handbook)
# antoine_coeffs = [A, B, C]
antoine_benzene = [6.90565, 1211.033, 220.79]
antoine_toluene = [6.95464, 1344.8,   219.482]
antoine_coeffs = np.array([antoine_benzene, antoine_toluene])

print("Parameters and component data are defined.")

In [ ]:
def calculate_K_values(T, P, antoine_coeffs):
    """Calculates VLE K-values for all components at a given T and P."""
    A = antoine_coeffs[:, 0]
    B = antoine_coeffs[:, 1]
    C = antoine_coeffs[:, 2]
    
    # Calculate saturated vapor pressure in mmHg
    P_sat_mmHg = 10**(A - (B / (C + T)))
    # Convert P_sat to atm (since P is in atm)
    P_sat_atm = P_sat_mmHg / 760.0
    
    # Calculate K-values using Raoult's Law
    K = P_sat_atm / P
    return K

In [ ]:
def rachford_rice_equation(psi, z, K):
    """The Rachford-Rice flash equation we want to solve.
       We want to find the value of psi that makes this function equal to zero.
    """
    return np.sum(z * (K - 1) / (1 + psi * (K - 1)))

# --- Perform the Simulation ---

# 1. Calculate K-values at the drum T and P
K = calculate_K_values(T, P, antoine_coeffs)

# 2. Solve the Rachford-Rice equation for the vapor fraction (psi)
# We need an initial guess for the solver. 0.5 is a safe guess.
psi_initial_guess = 0.5
psi_solution = fsolve(rachford_rice_equation, psi_initial_guess, args=(z, K))[0]

# 3. Check if the solution is physically meaningful (0 <= psi <= 1)
if psi_solution < 0 or psi_solution > 1:
    raise ValueError(f"Vapor fraction psi ({psi_solution:.3f}) is not between 0 and 1. The feed may be subcooled liquid or superheated vapor at these conditions.")

# 4. Calculate L, V, and the compositions x_i and y_i
V = psi_solution * F
L = F - V
x = z / (1 + psi_solution * (K - 1))
y = K * x

print("Simulation complete. All stream properties calculated.")

## Part 4: Analyzing the Design Results

Now that the simulation is complete, let's analyze the results to see if our design successfully separated the components.

In [ ]:
print("--- Flash Drum Simulation Results ---")
print(f"Operating Temperature: {T} C")
print(f"Operating Pressure:    {P} atm")
print("-" * 35)
print(f"Vapor Fraction (V/F):  {psi_solution:.3f}")
print(f"Vapor Flow Rate (V):   {V:.3f} mol/s")
print(f"Liquid Flow Rate (L):  {L:.3f} mol/s")
print("-" * 35)
print(f"Liquid Composition (x):")
print(f"  - Benzene: {x[0]:.3f} (mole fraction)")
print(f"  - Toluene: {x[1]:.3f} (mole fraction)")
print("-" * 35)
print(f"Vapor Composition (y):")
print(f"  - Benzene: {y[0]:.3f} (mole fraction)")
print(f"  - Toluene: {y[1]:.3f} (mole fraction)")
print("-" * 35)

# Sanity check: Do mole fractions sum to 1?
print(f"Sum of x_i: {np.sum(x):.4f}")
print(f"Sum of y_i: {np.sum(y):.4f}")

### Visualizing the Separation

A plot is the best way to see the results clearly.

In [ ]:
labels = ['Benzene', 'Toluene']
feed_comp = z
liquid_comp = x
vapor_comp = y

x_axis = np.arange(len(labels))
width = 0.25

plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))

rects1 = ax.bar(x_axis - width, feed_comp, width, label='Feed', color='gray')
rects2 = ax.bar(x_axis, liquid_comp, width, label='Liquid Product', color='royalblue')
rects3 = ax.bar(x_axis + width, vapor_comp, width, label='Vapor Product', color='firebrick')

ax.set_ylabel('Mole Fraction', fontsize=12)
ax.set_title('Composition of Streams', fontsize=16, weight='bold')
ax.set_xticks(x_axis)
ax.set_xticklabels(labels)
ax.legend()
ax.set_ylim(0, 1.0)

plt.show()

print("Interpretation: The vapor stream is enriched with Benzene (the more volatile component), and the liquid stream is enriched with Toluene. The separation was successful!")

## Part 5: Be the Process Engineer - What If?

The power of simulation is asking "what if?" without building a plant. Modify the **Flash Drum Operating Conditions** in the first code cell of Part 3 and re-run the simulation to find out!

#### Challenge 1: The Effect of Temperature
Increase the drum temperature `T` from 95 C to 105 C. 
*Before you run, predict*: Will more or less of the feed vaporize? How will this affect the purity of the benzene in the vapor stream? Why?

#### Challenge 2: The Effect of Pressure
Reset the temperature to 95 C. Now, decrease the drum pressure `P` from 1.2 atm to 1.0 atm (atmospheric pressure).
*Predict*: How does lowering the pressure affect the vaporization and the separation? Think about Raoult's Law and the K-values.